# <font style="color: rgb(50, 120, 229)"> Implementar una red neuronal en Esp32 </font>

La IA en el borde (Edge AI) es el proceso de ejecutar algoritmos de inteligencia artificial en dispositivos en el borde de Internet u otras redes. El enfoque tradicional para la IA y el aprendizaje automático es usar servidores potentes basados en la nube para realizar el entrenamiento del modelo así como la inferencia (predicciones).

Aunque los dispositivos en el borde pueden tener recursos limitados en comparación con sus homólogos basados en la nube, ofrecen una reducción en el uso del ancho de banda, menor latencia y una mayor privacidad de los datos.


<font style="color: rgb(50, 120, 229)"> Objectivos </font>

En esta práctica vamos a implementar una red neuronal en un ESP32, un microcontrolador de bajo costo y bajo consumo de energía. La red neuronal que vamos a implementar es un clasificador de poses de la mano, que puede clasificar entre 5 poses diferentes (arriba, abajo, izquierda, derecha y centro).

Entrenaremos el modelo utilizando los datos del sensor MPU6050 adquiridos en la práctica anterior.

### <font style="color: rgb(50, 120, 229)"> 1.1 Importar los datos </font>

Vamos a importar los datos que adquirimos en la práctica anterior utilizando la librería pandas.

In [31]:
import pandas as pd

data = pd.read_csv("./data/datos_mpu6050.csv")#TODO: Cargar el dataset usando pandas

#Mezclamos los datos para que no haya sesgo
data = data.sample(frac=1)

data.head()

,acelX,acelY,acelZ,output
1513,10.45,-0.73,0.01,DOWN
607,1.35,9.46,2.10,RIGHT
684,1.27,9.23,3.01,RIGHT
1648,9.22,0.26,-4.97,DOWN
1166,0.99,-9.88,0.40,LEFT


### <font style="color: rgb(50, 120, 229)"> 1.2 Preprocesar los datos </font>

Vamos a realizar los siguientes pasos para preprocesar los datos:

- Separar los datos en entrada y salida.
- Codificar las etiquetas de salida aplicando one-hot encoding.
- Separar los datos en entrenamiento y prueba.

In [32]:
#TODO: Crear un dataframe con las columnas acelX, acelY y acelZ, guardalo en la variable X
X = data[["acelX", "acelY", "acelZ"]]
X.head()

,acelX,acelY,acelZ
1513,10.45,-0.73,0.01
607,1.35,9.46,2.10
684,1.27,9.23,3.01
1648,9.22,0.26,-4.97
1166,0.99,-9.88,0.40


In [33]:
#TODO: Crear un dataframe con la columna output, guardalo en la variable y
y = data["output"]
y.head()

,output
1513,DOWN
607,RIGHT
684,RIGHT
1648,DOWN
1166,LEFT


Para codificar las etiquetas de salida vamos a utilizar la función `get_dummies` de pandas.

```python
import pandas as pd

encoded_labels = pd.get_dummies(labels, dtype=dtype)
```

- `labels`: es un arreglo de numpy o una serie de pandas con las etiquetas de salida.
- `dtype`: es el tipo de dato de las columnas de la matriz de salida (int, float, etc).

In [34]:
#TODO: Convertir la columna y en un one-hot encoding usando pd.get_dummies, recuerda especificar el tipo de dato como float
y = pd.get_dummies(y, dtype=float)
y.head()


,DOWN,IDLE,LEFT,RIGHT,TOP
1513,1.0,0.0,0.0,0.0,0.0
607,0.0,0.0,0.0,1.0,0.0
684,0.0,0.0,0.0,1.0,0.0
1648,1.0,0.0,0.0,0.0,0.0
1166,0.0,0.0,1.0,0.0,0.0


Un paso muy importante es separar los datos en entrenamiento y prueba, esto nos permitirá evaluar el modelo en datos que no ha visto durante el entrenamiento.

Realizaremos la separación utilizando la función `sample` de pandas.

```python
train_data = data.sample(frac=0.8)
test_data = data.drop(train_data.index)
```

- **frac**: es el porcentaje de datos que se utilizarán para entrenamiento. En este caso, el 80% de los datos se utilizarán para entrenamiento.

- **drop**: Elimina las filas con los índices especificados, en este caso, eliminamos las filas que se utilizaron para entrenamiento.

In [42]:
#TODO: Dividir los datos en entrenamiento y prueba usando X.sample y y.sample
# Guarda los datos de entrenamiento en las variables X_train y y_train
# Guarda los datos de prueba en las variables X_test y y_test
X_train = X.sample(frac=0.8, random_state=0)
X_test = X.drop(X_train.index)

y_train = y.sample(frac=0.8, random_state=0)
y_test = y.drop(y_train.index)

Por último, vamos a normalizar los datos de entrada utilizando la librería `pandas`.

La normalización que vamos a utilizar se conoce como `estandarización` y se calcula utilizando la siguiente fórmula:

$$
X_{std} = \frac{X - \mu}{\sigma}
$$

Donde:

- $X_{std}$: es el valor normalizado.
- $X$: es el valor original.
- $\mu$: es la media de los datos.
- $\sigma$: es la desviación estándar de los datos.

**La estandarización se aplica a todos los datos, pero el cálculo de la media y la desviación estándar se realiza solo en los datos de entrenamiento.**

In [43]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

print(f"MEAN: {mean.values}")
print(f"STD: {std.values}")

MEAN: [ 1.47187113 -0.27938042  3.00908922]
STD: [6.02016609 5.63723474 4.26806085]


**Los valores obtenidos en la celda anterior se utilizaran para normalizar los datos en la ESP32.**

In [44]:
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

X_train.head()

,acelX,acelY,acelZ
1124,-1.325856,-0.032040,0.937407
1674,1.376728,-0.152667,-1.433693
1001,-1.427182,-0.482616,0.672650
785,-1.691294,0.054882,-0.044303
1935,1.431543,0.274848,-0.674566


**Keras ya proporciona una capa de normalización que se puede utilizar en la red neuronal, no la utilizamos en este caso por que TF Lite Micro aun no soporta esta capa de Keras.**

Hasta este punto, hemos realizado todos los pasos necesarios para preprocesar los datos, pero tenemos los datos en un formato de pandas, necesitamos convertirlos a un formato que pueda ser utilizado por Keras.

Vamos a convertir los datos a un arreglo de numpy utilizando el atributo `values`.

```python
train_data = train_data.values
test_data = test_data.values
```

In [45]:
#TODO: Convertir los datos de entrenamiento y prueba a un arreglo de numpy usando .values
X_train = X_train.values
X_test = X_test.values
y_train = y_train.values
y_test = y_test.values

## <font style="color: rgb(50, 120, 229)"> 2. Crear el modelo </font>

Ya que hemos preprocesado los datos, vamos a crear el modelo de la red neuronal.

El modelo que vamos a crear es un modelo secuencial, que consta de las siguientes capas:

- Capa de entrada.
- Capa densa con 8 neuronas y función de activación ReLU.
- Capa de salida con 5 neuronas y función de activación softmax.

```python
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()

model.add(Input(shape=(3,)))
model.add(Dense(8, activation='relu'))
```

- `Input`: Capa de entrada.
- `Dense`: Capa densa.
- `shape`: Forma de los datos de entrada.
- `activation`: Función de activación.

In [46]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

#TODO: Crear un modelo secuencial
model = Sequential()

#TODO: Agregar una capa de entrada con 3 neuronas, una para cada columna de X
model.add(Input(shape=(3,)))

#TODO: Agregar una capa densa con 8 neuronas y activación relu
model.add(Dense(8, activation='relu'))

#TODO: Agregar una capa densa con 5 neuronas y activación softmax, 5 neuronas porque tenemos 5 clases en la salida
model.add(Dense(5, activation='softmax'))

model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_9 (Dense)                      │ (None, 8)                   │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 5)                   │              45 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 77 (308.00 B)

 Trainable params: 77 (308.00 B)

 Non-trainable params: 0 (0.00 B)

Después de definir la arquitectura de la red, vamos a compilar el modelo.

```python
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
```

- `optimizer`: Optimizador.
- `loss`: Función de pérdida, en este caso, utilizamos la entropía cruzada categórica porque estamos realizando una clasificación multiclase.
- `metrics`: Métricas que se utilizarán para evaluar el modelo.

In [47]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Por último, vamos a entrenar el modelo.

```python
model.fit(train_data, train_labels, epochs=20, validation_data=(test_data, test_labels))
```

- `epochs`: Número de épocas.
- `validation_data`: Datos de validación.


In [48]:
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - accuracy: 0.1585 - loss: 1.7272 - val_accuracy: 0.3812 - val_loss: 1.5329
Epoch 2/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.3956 - loss: 1.5063 - val_accuracy: 0.6485 - val_loss: 1.3359
Epoch 3/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.6186 - loss: 1.3153 - val_accuracy: 0.7005 - val_loss: 1.1424
Epoch 4/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7448 - loss: 1.1026 - val_accuracy: 0.9010 - val_loss: 0.9529
Epoch 5/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8988 - loss: 0.9177 - val_accuracy: 0.9505 - val_loss: 0.7792
Epoch 6/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9602 - loss: 0.7553 - val_accuracy: 0.9678 - val_loss: 0.6341
Epoch 7/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.9778 - loss: 0.6097 - val_accuracy: 0.9802 - val_loss: 0.5174
Epoch 8/20
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9904 - loss: 0.5000 - val_accuracy: 0.9926 - val_l

**Si el modelo se entrena correctamente debes de obtener un valor de perdida bajo y un valor de precisión alto, ádemas de que la precisión en los datos de validación debe ser similar a la precisión en los datos de entrenamiento.**

## <font style="color: rgb(50, 120, 229)"> 3. Convertir el modelo a TensorFlow Lite </font>

Una vez que hemos entrenado el modelo, vamos a convertirlo a TensorFlow Lite para poder ejecutarlo en la ESP32.

Para convertir el modelo a TensorFlow Lite, vamos a utilizar la función `TFLiteConverter` de TensorFlow.

In [49]:
from tensorflow import lite as tflite

model_name = "mpu6050_model" #Nombre del archivo donde se guardará el modelo

converter = tflite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert() #Convertimos el modelo a un modelo tflite

with open(f"{model_name}.tflite", 'wb') as f: #Abrimos un archivo en modo escritura binaria
    f.write(tflite_model) #Guardamos el modelo en un archivo llamado model.tflite

Saved artifact at '/tmp/tmp7ofzr28f'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 3), dtype=tf.float32, name='keras_tensor_19')
Output Type:
  TensorSpec(shape=(None, 5), dtype=tf.float32, name=None)
Captures:
  135366664022928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135366529165168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135366528102144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135366528100560: TensorSpec(shape=(), dtype=tf.resource, name=None)


Muchas plataformas de microcontroladores no tienen soporte para TensorFlow Lite. La forma más sencilla de ejecutar un modelo de TensorFlow Lite en un microcontrolador es convertirlo a una matriz de bytes y ejecutarlo en el microcontrolador.


In [50]:
def tflite_to_array(model_data, model_name):
    c_str = ""

    #Creamos las cabeceras del archivo
    c_str += f"#ifndef {model_name.upper()}_H\n"
    c_str += f"#define {model_name.upper()}_H\n\n"

    #Agregamos una variable con el tamaño del modelo
    c_str += f"const unsigned int {model_name}_len = {len(model_data)};\n\n"

    #Agregamos el modelo como un arreglo de bytes
    c_str += f"const unsigned char {model_name}[] = {{\n"

    for i, byte in enumerate(model_data):
        c_str += f"0x{byte:02X},"
        if (i + 1) % 12 == 0:
            c_str += "\n"

    c_str += "};\n\n"

    #Cerramos las cabeceras del archivo
    c_str += f"#endif // {model_name.upper()}_H\n"

    return c_str

In [51]:
model_array = tflite_to_array(tflite_model, model_name)

with open(f"{model_name}.h", 'w') as f:
    f.write(model_array)